In [41]:
import os, glob
path_files_texts = []
path_to_dir_of_path_reports = "/Users/tp53/Documents/code/rgci_kundan_chufal/sample_hpe"

# Loop through all the text files in the directory and store contents in a giant list
for file in glob.glob(os.path.join(path_to_dir_of_path_reports, "*.txt")):
    # Open each file and read its contents into a string
    with open(file, "r" , encoding='cp1252') as f:
        text_contents_of_file = f.read()
        # Append the string to the list
        path_files_texts.append(text_contents_of_file)

In [48]:
import os
print(os.getcwd())

/Users/tp53/Documents/code/rgci_kundan_chufal


In [49]:
import cohere
import numpy as np

apiKey = 'acYUhCGgSu54zqgZDFrrJtgLsNFYMF59L0992b6I'
co = cohere.Client(apiKey)

'''
embeds = co.embed(texts=path_files_texts,                  				
  					model="small",
  					truncate="START").embeddings

np.save('cohere_embeddings_small.npy', embeds)
'''


'\nembeds = co.embed(texts=path_files_texts,                  \t\t\t\t\n  \t\t\t\t\tmodel="small",\n  \t\t\t\t\ttruncate="START").embeddings\n\nnp.save(\'cohere_embeddings_small.npy\', embeds)\n'

In [53]:
import os
print(os.getcwd)

import numpy as np

embeds = np.load('cohere_embeddings_small.npy')

#import umap
import umap.umap_ as umap
import pandas as pd
import altair as alt


'''
import cohere
import numpy as np
import re

from tqdm import tqdm
from datasets import load_dataset


from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
from sklearn.cluster import KMeans
from bertopic._ctfidf import ClassTFIDF
from sklearn.feature_extraction.text import CountVectorizer

'''

reducer = umap.UMAP(n_neighbors=10) #50 as there are only 74 files
umap_embeds = reducer.fit_transform(embeds)

#df = pd.DataFrame(path_files_texts)
df = pd.DataFrame(path_files_texts, columns=["text"])

print(df.shape)

df['x'] = umap_embeds[:,0]
df['y'] = umap_embeds[:,1]

print(df.shape)


# Plot
chart = alt.Chart(df).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    tooltip=['text']
).configure(background="#FDF7F0"
).properties(
    width=700,
    height=400,
    title='Pathology reports'
)

chart.save('chart.html')
chart.interactive()


<built-in function getcwd>
(74, 1)
(74, 3)


alt.Chart(...)

In [54]:
#25april2023
#finish color coded, clusters with representative words using topically, bertopic today

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
#!pip install bertopic topically

from topically import Topically
#from bertopic import BERTopic
from sklearn.cluster import KMeans


# Load and initialize BERTopic to use KMeans clustering with 8 clusters only.
n_clusters = 8
kmeans_model = KMeans(n_clusters=n_clusters)
classes = kmeans_model.fit_predict(embeds)

print(classes.shape)

print(classes)
'''
cluster_model = KMeans(n_clusters=n_clusters)
topic_model = BERTopic(hdbscan_model=cluster_model)

# df is a dataframe. df['text'] is the column of text we're modeling
df['topic'], probabilities = topic_model.fit_transform(df['text'], embeds)
'''

(74,)
[6 1 5 1 6 1 1 1 7 1 2 2 0 2 1 6 1 2 1 1 2 2 1 0 3 3 1 1 6 1 1 2 1 2 1 3 3
 3 6 5 3 2 6 6 7 2 1 3 2 2 2 6 6 5 5 5 3 6 4 6 3 2 5 1 6 6 7 6 3 6 3 0 2 6]


"\ncluster_model = KMeans(n_clusters=n_clusters)\ntopic_model = BERTopic(hdbscan_model=cluster_model)\n\n# df is a dataframe. df['text'] is the column of text we're modeling\ndf['topic'], probabilities = topic_model.fit_transform(df['text'], embeds)\n"

In [61]:
#from bertopic.vectorizers import _ctfidf as ClassTFIDF
#from .vectorizers._ctfidf import ClassTFIDF
from bertopic.vectorizers._ctfidf import ClassTfidfTransformer as CTT
from sklearn.feature_extraction.text import CountVectorizer

# Extract the keywords for each cluster
documents =  df['text']
documents = pd.DataFrame({"Document": documents,
                          "ID": range(len(documents)),
                          "Topic": None})
documents['Topic'] = classes
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
count_vectorizer = CountVectorizer(stop_words="english").fit(documents_per_topic.Document)
count = count_vectorizer.transform(documents_per_topic.Document)
words = count_vectorizer.get_feature_names_out()
#vectorizer = CountVectorizer()
#words = vectorizer.get_feature_names()
ctfidf = CTT().fit_transform(count).toarray()
words_per_class = {label: [words[index] for index in ctfidf[label].argsort()[-10:]] for label in documents_per_topic.Topic}
df['cluster'] = classes
df['keywords'] = df['cluster'].map(lambda topic_num: ", ".join(np.array(words_per_class[topic_num])[:]))

In [62]:
df.head()


,text,x,y,cluster,keywords
0,BIOPSY NO: B/9791/2014\n\nSPECIMEN: CT GUIDED ...,6.722672,4.175910,6,"pd, measuring, soft, opinion, bits, microscopi..."
1,BIOPSY NO.: B/177/2014\n\nSPECIMEN SUBMITTED: ...,6.153565,7.439694,1,"free, carcinoma, left, 001, tissue, deposit, r..."
2,PAP NO.: P/3689/2018 CERVICAL SMEAR\n\nRECEIVE...,0.784490,4.991347,5,"unsatisfactory, adequacy, smear, evaluation, n..."
3,BIOPSY NO.: B/2619/2013\n\nSPECIMEN SUBMITTED:...,5.799022,6.713177,1,"free, carcinoma, left, 001, tissue, deposit, r..."
4,PND HISTOPATH 1-2 BLOCK PAYMENT\n\nBIOPSY NO. ...,6.872034,4.708961,6,"pd, measuring, soft, opinion, bits, microscopi..."
